# Binary Search

**Purpose:** In this article we will explore binary search in ocaml

In [26]:
#require "jupyter.notebook" ;;
(* #require "jupyter-archimedes" *)


## Problem Setup

For this problem we will be fed a direction, but we won't know the exact coordinate.  Here we create the functions that given a current location and a target location gives us the direction (ie DR, or D)

In [27]:
exception Damn of string

let get_width_dir bw cw = 
  match compare cw bw  with
    |0 -> ""|1 -> "L"|(-1) -> "R"| _ -> raise (Damn "error")

let get_height_dir bh ch = 
  match compare ch bh with
    |0 -> "" |1 -> "U" |(-1) -> "D"| _ -> raise (Damn "error")

let get_direction currentloc bombloc = 
  let bw,bh = bombloc in
  let ((cw,ch),(_,_,_,_)) = currentloc in

  (get_height_dir bh ch) ^ (get_width_dir bw cw)

exception Damn of string


val get_width_dir : 'a -> 'a -> string = <fun>


val get_height_dir : 'a -> 'a -> string = <fun>


val get_direction : ('a * 'b) * ('c * 'd * 'e * 'f) -> 'a * 'b -> string =
  <fun>


## Step Function

Next we need to write out step function.  Every step is to the center of the available board, and we shrink the available board with each step as we are given the location.

In [28]:
let take_step currentloc dir =
  let ((cw,ch),(w0,w1,h0,h1)) = currentloc in
  match dir with 
        |"L" -> (((w0+cw)/2,ch),(w0,cw,ch,ch))
        |"R" -> (((cw+w1)/2,ch),(cw,w1,ch,ch))
        |"U" -> ((cw,(h0+ch)/2),(cw,cw,h0,ch))
        |"D" -> ((cw,(ch+h1)/2),(cw,cw,ch,h1))
        |"UL" -> (((w0+cw)/2,(h0+ch)/2),(w0,cw,h0,ch))
        |"UR" -> (((cw+w1)/2,(w0+ch)/2),(cw,w1,h0,ch))
        |"DL" -> (((w0+cw)/2,(ch+h1)/2),(w0,cw,ch,h1))
        |"DR" -> (((cw+w1)/2,(ch+h1)/2),(cw,w1,ch,h1))
        |_ -> raise (Damn "error")

val take_step :
  (int * int) * (int * int * int * int) ->
  string -> (int * int) * (int * int * int * int) = <fun>


## The Test Loop

We will create a test loop that will iterate through until it finds a solution, with some printing so we can see what it's doing.

In [29]:
let print_step bl cl bd =
    let ((ow,oh),(bw1,bw2,bh1,bh2)) = cl in
    let bw, bh = bl in
    print_endline ("Board: " ^ string_of_int bw1 ^ "x"^ string_of_int bw2 ^ "x"^ string_of_int bh1 ^ "x"^ string_of_int bh2 ^ "  |  " ^
              "Bomb: " ^ string_of_int bw ^ "x" ^ string_of_int bh ^ "  |  " ^ 
              "Loc: " ^ string_of_int ow ^ "x" ^ string_of_int oh ^ "  |  " ^ 
              "Direction: " ^ bd);

val print_step :
  int * int -> (int * int) * (int * int * int * int) -> string -> unit =
  <fun>


In [30]:
let rec loop loc acc printf bl=
    let bombdir = get_direction loc bl in
    printf bl loc bombdir;

    let out = take_step loc bombdir in
    let ((ow,oh),(bw1,bw2,bh1,bh2)) = out in


    if (ow,oh)=(bl) then "success" 
    else if acc > 200 then "200 iters"
    else loop out (acc+1) printf bl

val loop :
  (int * int) * (int * int * int * int) ->
  int ->
  (int * int -> (int * int) * (int * int * int * int) -> string -> 'a) ->
  int * int -> string = <fun>


## The Expirament

We can see that even when run with a pretty large random size and locations setup, we solve the problem in relatively few steps.

In [33]:
let sz = 1_000_000
let w,h = (Random.int sz,Random.int sz)
let current_loc = (Random.int w,Random.int h)
let bomb_loc = (Random.int w,Random.int h)
let board = (0,w,0,h)
let current_loc = (current_loc,board)

val sz : int = 1000000


val w : int = 443358
val h : int = 838988


val current_loc : int * int = (280535, 87436)


val bomb_loc : int * int = (363867, 38701)


val board : int * int * int * int = (0, 443358, 0, 838988)


val current_loc : (int * int) * (int * int * int * int) =
  ((280535, 87436), (0, 443358, 0, 838988))


In [34]:
loop current_loc 0 print_step bomb_loc

Board: 0x443358x0x838988  |  Bomb: 363867x38701  |  Loc: 280535x87436  |  Direction: UR
Board: 280535x443358x0x87436  |  Bomb: 363867x38701  |  Loc: 361946x43718  |  Direction: UR
Board: 361946x443358x0x43718  |  Bomb: 363867x38701  |  Loc: 402652x162126  |  Direction: UL
Board: 361946x402652x0x162126  |  Bomb: 363867x38701  |  Loc: 382299x81063  |  Direction: UL
Board: 361946x382299x0x81063  |  Bomb: 363867x38701  |  Loc: 372122x40531  |  Direction: UL
Board: 361946x372122x0x40531  |  Bomb: 363867x38701  |  Loc: 367034x20265  |  Direction: DL
Board: 361946x367034x20265x40531  |  Bomb: 363867x38701  |  Loc: 364490x30398  |  Direction: DL
Board: 361946x364490x30398x40531  |  Bomb: 363867x38701  |  Loc: 363218x35464  |  Direction: DR
Board: 363218x364490x35464x40531  |  Bomb: 363867x38701  |  Loc: 363854x37997  |  Direction: DR
Board: 363854x364490x37997x40531  |  Bomb: 363867x38701  |  Loc: 364172x39264  |  Direction: UL
Board: 363854x364172x37997x39264  |  Bomb: 363867x38701  |  Loc: 3

- : string = "success"
